In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from certifia.data_engineering.data_access import read_db

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
datasets = read_db()

In [3]:
datasets.keys()

dict_keys(['batch1', 'batch2', 'test'])

In [4]:
df_batch1 = datasets['batch1']
df_batch2 = datasets['batch2']
df_test = datasets['test']

## Définition de nos labels et de nos features

In [5]:
FEATURES = df_test['vols'].columns.tolist()
FEATURES

['IDENTIFIANT',
 'VOL',
 'CODE AVION',
 'AEROPORT DEPART',
 'AEROPORT ARRIVEE',
 'DEPART PROGRAMME',
 'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE',
 'TEMPS PROGRAMME',
 'DISTANCE',
 "TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE",
 'ARRIVEE PROGRAMMEE',
 'COMPAGNIE AERIENNE',
 'NOMBRE DE PASSAGERS',
 'DATE',
 'NIVEAU DE SECURITE']

In [6]:
LABELS = list(set(df_batch1['vols'].columns.tolist()) - set(df_test['vols'].columns.tolist()))
LABELS

['ANNULATION',
 'HEURE DE DEPART',
 'DECOLLAGE',
 'TEMPS PASSE',
 'RETARD COMPAGNIE',
 "RAISON D'ANNULATION",
 'RETARD METEO',
 'RETARD AVION',
 'ATTERRISSAGE',
 'RETARD SYSTEM',
 'TEMPS DE VOL',
 "RETARD A L'ARRIVEE",
 'RETARD SECURITE',
 'RETART DE DEPART',
 "HEURE D'ARRIVEE",
 'DETOURNEMENT']

In [9]:
df_batch1.keys(), df_batch2.keys(), df_test.keys()

(dict_keys(['vols', 'aeroports', 'compagnies', 'prix_fuel']),
 dict_keys(['vols', 'aeroports', 'compagnies', 'prix_fuel']),
 dict_keys(['vols']))

# Feature Engineering

In [101]:
df_batch1['vols'].head()

,IDENTIFIANT,VOL,CODE AVION,AEROPORT DEPART,AEROPORT ARRIVEE,DEPART PROGRAMME,HEURE DE DEPART,RETART DE DEPART,TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE,DECOLLAGE,TEMPS PROGRAMME,TEMPS PASSE,TEMPS DE VOL,DISTANCE,ATTERRISSAGE,TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE,ARRIVEE PROGRAMMEE,HEURE D'ARRIVEE,RETARD A L'ARRIVEE,DETOURNEMENT,ANNULATION,RAISON D'ANNULATION,RETARD SYSTEM,RETARD SECURITE,RETARD COMPAGNIE,RETARD AVION,RETARD METEO,COMPAGNIE AERIENNE,NOMBRE DE PASSAGERS,DATE,NIVEAU DE SECURITE
0,1259209,4661,a02782cd75,CEB,AAL,1707,1658.0,-9.0,20.0,1718.0,67.0,71.0,45.0,232,1803.0,6.0,1814,1809.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,MAF,379,15/8/2018,10
1,4886177,5026,707f6ea54f,GOI,LTK,600,553.0,-7.0,11.0,604.0,130.0,119.0,91.0,738,835.0,17.0,910,852.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN,I6F,9,2/11/2016,10
2,183332,2021,b116987956,DSS,JNB,1749,1747.0,-2.0,9.0,1756.0,248.0,228.0,215.0,1671,1831.0,4.0,1857,1835.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN,NVPPA,2491,9/6/2017,10
3,937517,1320,a4b8db63f5,AGP,GOA,2301,2322.0,21.0,19.0,2341.0,65.0,89.0,59.0,214,40.0,11.0,6,51.0,45.0,0,0,None,24.0,0.0,0.0,6.0,15.0,NVPPA,1241,26/5/2018,10
4,2157498,508,34604053c0,BRU,BOD,612,603.0,-9.0,13.0,616.0,302.0,259.0,238.0,2288,1314.0,8.0,1414,1322.0,-52.0,0,0,None,NaN,NaN,NaN,NaN,NaN,THA,78,10/11/2018,10


In [102]:
df_batch1['vols'].describe(include='all')

,IDENTIFIANT,VOL,CODE AVION,AEROPORT DEPART,AEROPORT ARRIVEE,DEPART PROGRAMME,HEURE DE DEPART,RETART DE DEPART,TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE,DECOLLAGE,TEMPS PROGRAMME,TEMPS PASSE,TEMPS DE VOL,DISTANCE,ATTERRISSAGE,TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE,ARRIVEE PROGRAMMEE,HEURE D'ARRIVEE,RETARD A L'ARRIVEE,DETOURNEMENT,ANNULATION,RAISON D'ANNULATION,RETARD SYSTEM,RETARD SECURITE,RETARD COMPAGNIE,RETARD AVION,RETARD METEO,COMPAGNIE AERIENNE,NOMBRE DE PASSAGERS,DATE,NIVEAU DE SECURITE
count,3.000000e+06,3.000000e+06,3000000,3000000,3000000,3.000000e+06,2.950817e+06,2.950817e+06,2.948974e+06,2.948974e+06,2.999997e+06,2.940503e+06,2.940503e+06,3.000000e+06,2.947215e+06,2.947215e+06,3.000000e+06,2.947215e+06,2.940503e+06,3.000000e+06,3.000000e+06,51528,559953.000000,559953.000000,559953.000000,559953.000000,559953.000000,3000000,3.000000e+06,3000000,3000000.0
unique,NaN,NaN,4189,318,318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,13,NaN,1002,NaN
top,NaN,NaN,786a02f742,AGP,AGP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NVPPA,NaN,26/6/2016,NaN
freq,NaN,NaN,9360,221117,221002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28044,NaN,NaN,NaN,NaN,NaN,575040,NaN,3408,NaN
mean,2.616680e+06,2.272942e+03,NaN,NaN,NaN,1.328994e+03,1.333179e+03,9.436509e+00,1.726023e+01,1.355807e+03,1.459086e+02,1.417271e+02,1.166868e+02,8.465176e+02,1.471460e+03,7.792886e+00,1.493786e+03,1.477092e+03,4.901089e+00,2.656333e-03,1.717600e-02,NaN,15.381552,0.085525,19.693153,22.738483,3.173741,NaN,4.590983e+02,NaN,10.0
std,1.727235e+06,1.824356e+03,NaN,NaN,NaN,4.855726e+02,4.981603e+02,3.921812e+01,9.281802e+00,5.003703e+02,7.911707e+01,7.818913e+01,7.639179e+01,6.439932e+02,5.200030e+02,5.988985e+00,5.052654e+02,5.241305e+02,4.148457e+01,5.147114e-02,1.299269e-01,NaN,30.300445,2.291357,51.342073,44.171470,21.909615,NaN,6.720171e+02,NaN,0.0
min,2.000000e+00,1.000000e+00,NaN,NaN,NaN,1.000000e+00,1.000000e+00,-6.100000e+01,1.000000e+00,1.000000e+00,1.800000e+01,1.500000e+01,7.000000e+00,3.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,-1.000000e+00,NaN,10.0
25%,1.043596e+06,7.770000e+02,NaN,NaN,NaN,9.160000e+02,9.200000e+02,-5.000000e+00,1.200000e+01,9.350000e+02,8.800000e+01,8.500000e+01,6.000000e+01,3.700000e+02,1.055000e+03,4.000000e+00,1.110000e+03,1.100000e+03,-1.400000e+01,0.000000e+00,0.000000e+00,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,7.700000e+01,NaN,10.0
50%,2.089660e+06,1.728000e+03,NaN,NaN,NaN,1.325000e+03,1.329000e+03,-2.000000e+00,1.500000e+01,1.342000e+03,1.250000e+02,1.220000e+02,9.600000e+01,6.660000e+02,1.508000e+03,6.000000e+00,1.520000e+03,1.512000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,NaN,4.000000,0.000000,1.000000,0.000000,0.000000,NaN,2.420000e+02,NaN,10.0
75%,4.288415e+06,3.410000e+03,NaN,NaN,NaN,1.730000e+03,1.738000e+03,7.000000e+00,2.000000e+01,1.752000e+03,1.790000e+02,1.750000e+02,1.480000e+02,1.089000e+03,1.911000e+03,9.000000e+00,1.917000e+03,1.916000e+03,8.000000e+00,0.000000e+00,0.000000e+00,NaN,20.000000,0.000000,19.000000,28.000000,0.000000,NaN,3.810000e+02,NaN,10.0


In [103]:
df_batch1['vols'][FEATURES+["RETARD A L'ARRIVEE"]].corr()

,IDENTIFIANT,VOL,DEPART PROGRAMME,TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE,TEMPS PROGRAMME,DISTANCE,TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE,ARRIVEE PROGRAMMEE,NOMBRE DE PASSAGERS,NIVEAU DE SECURITE,RETARD A L'ARRIVEE
IDENTIFIANT,1.000000,0.208508,0.000679,-0.028722,-0.165377,-0.165195,0.085273,-0.003014,-0.509965,NaN,0.042344
VOL,0.208508,1.000000,0.000844,0.026337,-0.391776,-0.401004,-0.037392,-0.012705,-0.176990,NaN,0.023328
DEPART PROGRAMME,0.000679,0.000844,1.000000,0.017896,-0.001466,0.004170,-0.050699,0.689619,0.002116,NaN,0.079363
TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE,-0.028722,0.026337,0.017896,1.000000,0.087760,0.054074,-0.030561,0.034437,0.030647,NaN,0.242590
TEMPS PROGRAMME,-0.165377,-0.391776,-0.001466,0.087760,1.000000,0.985256,0.081706,0.032009,0.033028,NaN,-0.023557
DISTANCE,-0.165195,-0.401004,0.004170,0.054074,0.985256,1.000000,0.061389,0.023366,0.029755,NaN,-0.021176
TEMPS DE DEPLACEMENT A TERRE A L'ATTERRISSAGE,0.085273,-0.037392,-0.050699,-0.030561,0.081706,0.061389,1.000000,-0.012797,-0.035866,NaN,0.120962
ARRIVEE PROGRAMMEE,-0.003014,-0.012705,0.689619,0.034437,0.032009,0.023366,-0.012797,1.000000,0.012521,NaN,0.073245
NOMBRE DE PASSAGERS,-0.509965,-0.176990,0.002116,0.030647,0.033028,0.029755,-0.035866,0.012521,1.000000,NaN,-0.040094
NIVEAU DE SECURITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = preprocessing.LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [232]:
class FeatureEngineering:
    def __init__(self, training_columns = None):
        self.training_columns = training_columns
    
    def cleaning(self, df):
        df = df.dropna(subset=["RETARD A L'ARRIVEE"])
        
        df = df.drop(columns=['NIVEAU DE SECURITE'])
        return df
    
    def split_feature_label(self, df):
        X = df[self.training_columns]
        y = df["RETARD A L'ARRIVEE"]
        return X, y
    
    def train_test_split_80_20(self, X, y): 
        return train_test_split(X, y, test_size=0.2, random_state=42)
    
    def dummify_columns(self, X, columns = None):
        return MultiColumnLabelEncoder(columns = columns).fit_transform(X)
        
    def transform(self, dataframe: pd.DataFrame):
        df = dataframe.copy()
        
        df = self.cleaning(df)
        
        X, y = self.split_feature_label(df)
        
        X = self.dummify_columns(X, ['AEROPORT DEPART','AEROPORT ARRIVEE'])
        
        return self.train_test_split_80_20(X, y)

In [233]:
vol_batch1_smaller = df_batch1['vols']#.head(1000000)

In [234]:
vol_batch1_with_retard_arrivee = vol_batch1_smaller.dropna(subset=["RETARD A L'ARRIVEE"])

In [235]:
X_batch1 = vol_batch1_with_retard_arrivee[['AEROPORT DEPART','AEROPORT ARRIVEE']]
y_batch1 = vol_batch1_with_retard_arrivee["RETARD A L'ARRIVEE"]

In [236]:
# dummification
#X_batch1_dummified = pd.concat([pd.get_dummies(X_batch1['AEROPORT DEPART'], prefix='AEROPORT_DEPART'),
#                               pd.get_dummies(X_batch1['AEROPORT ARRIVEE'], prefix='AEROPORT_ARRIVEE')], axis=1)

# Label encoder
X_batch1_dummified = MultiColumnLabelEncoder(
    columns = ['AEROPORT DEPART','AEROPORT ARRIVEE']
).fit_transform(X_batch1)

In [237]:
X_train, X_test, y_train, y_test = train_test_split(X_batch1_dummified, y_batch1, test_size=0.2, random_state=42)

In [238]:
X_train, X_test, y_train, y_test = FeatureEngineering(
        training_columns=['AEROPORT DEPART',
                          'AEROPORT ARRIVEE', 
                          #'TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE',
                          'NOMBRE DE PASSAGERS']
    ).transform(vol_batch1_smaller)

# Training

In [239]:
rf_regressor = RandomForestRegressor(n_estimators=30, max_depth=4, random_state=0, n_jobs=-1)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, n_estimators=50, n_jobs=-1, random_state=0)

# Prédictions

In [240]:
X_test.head(2)

,AEROPORT DEPART,AEROPORT ARRIVEE,NOMBRE DE PASSAGERS
1531238,7,309,65
2478044,212,7,1742


In [241]:
y_pred = rf_regressor.predict(X_test)

In [242]:
y_pred[:10]

array([7.75639378, 0.04761413, 4.0633513 , 5.0895171 , 5.62902844,
       4.09035192, 2.71076693, 4.0633513 , 4.91997576, 5.62902844])

In [243]:
y_test[:10]

1531238    -2.0
2478044   -29.0
2068905     1.0
919696     -3.0
2022056   -16.0
2451534   -12.0
1666013    -3.0
2114520     3.0
1576245   -10.0
2858069    -5.0
Name: RETARD A L'ARRIVEE, dtype: float64

# Evaluation du model

In [244]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 22.099588377937426
Mean Squared Error: 1667.3265462566103
Root Mean Squared Error: 40.83291008802349


# Save the model

In [245]:
import pickle
# Save to file in the current working directory
pkl_filename = "../data/output/rf_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(rf_regressor, file)

In [246]:
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
# Calculate the accuracy score and predict target values
score = pickle_model.score(X_test, y_test)
print("R2 score: {0:.2f}".format(score))
y_pred = pickle_model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R2 score: 0.01
Mean Absolute Error: 22.099588377937426
Mean Squared Error: 1667.3265462566103
Root Mean Squared Error: 40.83291008802349
